In [1]:
import os

from rio_glui.server import TileServer
from rio_glui.raster import RasterTiles

from mapboxgl.utils import *
from mapboxgl.viz import *

In [2]:
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [3]:
file = 'https://oin-hotosm.s3.amazonaws.com/5ac626e091b5310010e0d482/0/5ac626e091b5310010e0d483.tif'

In [4]:
import mercantile
import numpy

import cv2
from scipy.ndimage.morphology import binary_dilation
from rasterio.plot import reshape_as_image
from rio_tiler.utils import tile_read


# https://github.com/mapbox/rio-glui/blob/master/rio_glui/raster.py#L148-L158
class customRaster(RasterTiles):
    def read_tile(self, z, x, y):
        """Read raster tile data and mask."""
        mercator_tile = mercantile.Tile(x=x, y=y, z=z)
        tile_bounds = mercantile.xy_bounds(mercator_tile)

        data, mask = tile_read(
            self.path,
            tile_bounds,
            self.tiles_size,
            indexes=self.indexes,
            nodata=self.nodata,
        )

        #################################
        # apply your custom function here
        # Extract Linear Features
        # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html
        img = reshape_as_image(data)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 100)

        # Edges dilation for visual
        edges = binary_dilation(edges).astype(numpy.uint8) * 255
        
        data[0, edges > 0] = 255
        
        return data, mask

In [5]:
raster = customRaster(file, indexes=(1,2,3))
ts = TileServer(raster)
ts.start()

In [6]:
viz = RasterTilesViz(ts.get_tiles_url(), 
                     tiles_bounds=ts.get_bounds(),
                     center=ts.get_center(),
                     access_token=token, 
                     height='1000px', 
                     zoom=13)
viz.show()

In [8]:
ts.stop()